# DML

In [0]:
data = [(6,110,'Ansh', "JHB"),(7,420,'fred', "BHS"),(8,410,'Johnny', "BHS"),(9,420,'Thomas', "JHB"),(10,250,'Ronaldo', "CPT")]

df = spark.createDataFrame(data, ['id', 'salary', 'name'])

df.write.format("delta")\
    .mode("append")\
    .save("/Volumes/deltalakedidi/default/deltavolume/DMLSINK/")


In [0]:
%sql
SELECT * FROM delta.`/Volumes/deltalakedidi/default/deltavolume/DMLSINK/`

id,salary,name,_4
1,910,Bongi,JHB
2,600,Godfrey,PTA
3,400,Jane,BHS
4,420,Thabo,JHB
5,250,Steve,CPT
6,630,Ansh,JHB
7,420,fred,BHS
8,410,Johnny,BHS
9,420,Thomas,JHB
10,250,Ronaldo,CPT


## _UPDATE _

In [0]:
%sql
UPDATE delta.`/Volumes/deltalakedidi/default/deltavolume/DMLSINK/`
set salary = 630 where id = 6

num_affected_rows
1


### DELETION VECTORS

#### UPSERT


In [0]:
data = [(2,110,"Dima", "JHB"),(11,420,'Vivi', "BHS"),(8,410,'Johnny', "BHS"),(9,420,'Thomas', "JHB"),(10,250,'Ronaldo', "CPT")]

df = spark.createDataFrame(data, ['id', 'salary', 'name'])

display(df)

id,salary,name,_4
2,110,Dima,JHB
11,420,Vivi,BHS
8,410,Johnny,BHS
9,420,Thomas,JHB
10,250,Ronaldo,CPT


In [0]:
from delta.tables import DeltaTable

In [0]:
dlt_obj = DeltaTable.forPath(spark, "/Volumes/deltalakedidi/default/deltavolume/DMLSINK/")

dlt_obj.alias("trg").merge(df.alias("src"), "trg.id = src.id")\
    .whenMatchedUpdateAll()\
    .whenNotMatchedInsertAll()\
    .execute()

DataFrame[num_affected_rows: bigint, num_updated_rows: bigint, num_deleted_rows: bigint, num_inserted_rows: bigint]

In [0]:
%sql
SELECT * FROM delta.`/Volumes/deltalakedidi/default/deltavolume/DMLSINK/`
order by id

id,salary,name,_4
1,910,Bongi,JHB
2,110,Dima,JHB
3,400,Jane,BHS
4,420,Thabo,JHB
5,250,Steve,CPT
6,630,Ansh,JHB
7,420,fred,BHS
8,410,Johnny,BHS
9,420,Thomas,JHB
10,250,Ronaldo,CPT
